In [ ]:
!pip install polyleven

import pandas as pd
import numpy as np
from polyleven import levenshtein
from collections import Counter

data=pd.read_csv('DATASET/spikeprot0329_England.csv',index_col=False)
data=data.drop(columns='Unnamed: 0')
#[len(x) for x in data['Aligned'].values]
data

mask=[True if len(x)==1273 else False for x in data['Aligned'].values]
data=data[mask]

def expand_matrix(matrix,weights):
    for i,peso in enumerate(weights):
        if peso>1:
            
            red=np.copy(peso-1)
            #print(red)
            n=len(matrix)
            new_col=np.zeros(n)
            new_row=np.zeros(n+red)
            new_col=np.copy(matrix[:,i])
            new_row[:n]=np.copy(new_col)
            
            new_col=np.reshape(new_col,(len(new_col),1))*np.ones((n,red))
            
            new_row=(new_row)*np.ones((red,n+red))
            
            matrix = np.hstack((matrix, np.atleast_2d(new_col)))
            
            matrix=np.vstack((matrix,np.atleast_2d(new_row)))
            
            
            
    return matrix



for month in range(3,max(data['Month'])):
    mask=data['Month']==month
    temp=data[mask]
    temp=temp.reset_index()
    seq=list(set(temp['Sequence'].values))
    seq=pd.Series(seq)
    seq.to_csv(f'DISTANCE MATRICES/{month}_seq_list.csv')
    n=len(seq)
    matrix=np.zeros((n,n))
    for i in range(n):
        for k in range(i+1,n):
            dist=np.copy(levenshtein(seq[i],seq[k]))
            matrix[i][k]=dist
            matrix[k][i]=dist
    print(f'{month} at 1/2')
    diz_seq=Counter(temp['Sequence'].values)
    weights=[]
    for el in seq:
        weights.append(diz_seq[el])
    matrix=expand_matrix(matrix,weights)
    out_path=f'DISTANCE MATRICES/{month}' 
    np.save(out_path,matrix)
    print(f'{month} complete!')
    
